# Movies - Financial Data
- Andrea Cohen
- 03.01.23


## Business Problem:
 - to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset
 - to use this database to analyze what makes a movie successful
 - to provide recommendations to the stakeholder on how to make a successful movie

### Financial Data Problem:
- There is no financial information included in the IMDB data (e.g. budget or revenue).
- The stakeholder identified The Movie Database (TMDB) as a source of financial data (https://www.themoviedb.org/).
- The stakeholder wants the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

## Tasks:
- Download several files from IMDB’s movie data set and filter out the subset of moves requested by the stakeholder.
- Use an API to extract box office revenue and profit data to add to the IMDB data and perform exploratory data analysis.
- Construct and export a MySQL database using the data.
- Apply hypothesis testing to explore what makes a movie successful.
- Produce a Linear Regression model to predict movie performance.

### Financial Data Tasks:
- perform a test extraction of movies that started in 2000 or 2001
- each year should be saved as a separate .csv.gz file

## Data:

Data Location - The dataset files can be accessed and downloaded from https://datasets.imdbws.com/. The data is refreshed daily.

Data Source - TMDB

![png](TMDB1024_1.png)

IMDb Dataset Details -

- title.akas.tsv.gz -  
Contains the following information for titles:

 - titleId (string) - a tconst, an alphanumeric unique identifier of the title
 - ordering (integer) – a number to uniquely identify rows for a given titleId
 - title (string) – the localized title
 - region (string) - the region for this version of the title
 - language (string) - the language of the title
 - types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
 - attributes (array) - Additional terms to describe this alternative title, not enumerated
 - isOriginalTitle (boolean) – 0: not original title; 1: original title  
 
 
- title.basics.tsv.gz -   
Contains the following information for titles:
 - tconst (string) - alphanumeric unique identifier of the title
 - titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
 - primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
 - originalTitle (string) - original title, in the original language
 - isAdult (boolean) - 0: non-adult title; 1: adult title
 - startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
 - endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
 - runtimeMinutes – primary runtime of the title, in minutes
 - genres (string array) – includes up to three genres associated with the title   
 
 
- title.ratings.tsv.gz –   
Contains the IMDb rating and votes information for titles
 - tconst (string) - alphanumeric unique identifier of the title
 - averageRating – weighted average of all the individual user ratings
 - numVotes - number of votes the title has received

## Preliminary steps

### Imports

In [1]:
import pandas as pd
import numpy as np
import json, os, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

### Credentials and Accessing the API

In [2]:
with open('/Users/andreacohen/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
tmdb.API_KEY = login['api-key']

### Custom functions

In [3]:
# add the certification (MPGG Rating) to movie.info
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    #save the .info, .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    #loop through countries in releases
    for c in releases['countries']:
        #if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            #save a certification key in info with the certification
            info['certification'] = c['certification']
    return info

In [4]:
# append/extend a JSON file with Python
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    with open(filename,'r+') as file:
        # First load existing data into a dict
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Set file's current position as offset
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

### Confirm the API function works

#### Test the function on movie id: tt0848228 ("The Avengers")

In [5]:
test1 = get_movie_with_rating("tt0848228")
test1

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 146.526,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

- The API function returned the budget, revenue, and MPAA Rating for The Avengers.

#### Test the function on movie id: tt0332280 ("The Notebook")

In [6]:
test2 = get_movie_with_rating("tt0332280")
test2

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 55.267,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

- The API function returned the budget, revenue, and MPAA Rating for The Notebook.

## Before the loops


### Designate a folder to save the information

In [7]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

### Load in the Title Basics data

In [8]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

### Define the years to retrieve

In [9]:
YEARS_TO_GET = [2000,2001]

### Define an errors list

In [10]:
errors = [ ]

## Create a nested loop
### Outer loop organizes output by year into separate .json files
### Inner loop makes API calls based on the existing IMDB IDs

In [11]:
# set up progress bar
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # define a JSON_FILE to save the results in progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # if it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # define/filter the IDs to call
    # save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # save movie ids to list
    movie_ids = df['tconst'].copy()
    # load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # iterate through the list of Movie IDs and make the calls
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
    # save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
            
    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1433 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1548 [00:00<?, ?it/s]

## Print a message reporting back the number of movie ids that caused an error.

In [12]:
print(f"- Total errors: {len(errors)}")

- Total errors: 458
